In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
pd.options.display.max_columns = 999
import myLogFun
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
import lightgbm as lgb
from sklearn.preprocessing import PolynomialFeatures

In [2]:
clustered_df = pd.read_csv('../files/cluster_df.csv')
coor_df = clustered_df[['Latitude','Longitude']]
clustered_df = clustered_df.drop(['Latitude','Longitude','color'],axis=1)
clustered_df_log,log_tran_col = myLogFun.log_tran(clustered_df.copy(),['Total Traffic Volume (MB)','cluster_id'])
clustered_df_log.head()

,ln_Number of Subscribers,ln_Average DL Throughput (kbit/s),Average UL Throughput (kbit/s),ln_DL IBLER (%),ln_UL IBLER (%),DL MCS (only for Initial Transmission),UL MCS (only for Initial Transmission),ln_UL Spectrum Efficiency (bps/HZ),RSRP(All MRs) (dBm),ln_MR Count,Total Traffic Volume (MB),cluster_id
0,3.258097,8.668722,1023.061,2.604170,3.148882,8.0,17.0,-1.427116,-115.06,5.313206,67.130,0
1,3.912023,8.508687,1409.271,2.730464,3.065725,8.0,15.0,-1.171183,-114.87,6.416732,218.492,0
2,3.044522,8.615236,1471.403,2.679651,3.360723,7.0,16.0,-1.272966,-114.08,5.429346,109.247,0
3,0.693147,8.216814,865.221,2.989211,2.311545,5.0,16.0,-2.207275,-113.83,4.795791,3.864,0
4,3.610918,8.487582,1314.256,2.632608,3.216473,7.0,12.0,-1.347074,-113.01,5.981414,144.107,0


In [4]:
cluster_cnt_df = pd.DataFrame(clustered_df_log['cluster_id'].value_counts()).reset_index()

In [5]:
cluster_cnt_df.columns = ['cluster id','total grid count']

In [6]:
cluster_cnt_df.sort_values(by='cluster id').reset_index(drop=True)

,cluster id,total grid count
0,0,53824
1,1,54712
2,2,82274
3,3,39018
4,4,41047
5,5,157
6,6,54270


compute the baseline with only one model

In [7]:
# defining the variable
X = clustered_df_log.drop(['Total Traffic Volume (MB)','cluster_id'],axis=1).values
y = clustered_df_log['Total Traffic Volume (MB)'].values

# scaling
scX = StandardScaler()
X = scX.fit_transform(X)
scy = StandardScaler()
y = scy.fit_transform(y.reshape(-1,1))

# split data
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3, random_state=42)

using boosting method

In [14]:
# defining the model
model_lgb = lgb.LGBMRegressor(learning_rate=0.05,n_estimators=2000,num_leaves=100)
model_lgb.fit(X_train,y_train)

# predict and test the model
y_pred = model_lgb.predict(X_test)
r2_score(y_test,y_pred)

0.46162083812109667

more baseline with decision tree regeression

In [8]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import export_graphviz

In [9]:
tree_regressor = DecisionTreeRegressor(min_samples_leaf=10,min_samples_split=5,max_depth=10)
tree_regressor.fit(X_train,y_train)

# predict and test the model
y_pred = tree_regressor.predict(X_test)
r2_score(y_test,y_pred)

0.35183631957213934

In [10]:
export_graphviz(tree_regressor, out_file='tree.dot', feature_names = clustered_df_log.columns[:-2],
                rounded = True, proportion = False, precision = 2, filled = True)

In [12]:
!dot -Tpng tree.dot -o tree_limited.png -Gdpi=600

In [ ]:
from IPython.display import Image
Image(filename = 'tree_limited.png')

separate model for each cluster

In [3]:
c_df_log_list = []
for i in range(7):
    c_df_log_list.append(clustered_df_log[clustered_df_log['cluster_id']==i])

In [4]:
X_list = []
y_list = []
scX_list = []
scy_list = []
X_train_list = []
X_test_list = []
y_train_list = []
y_test_list = []

for i in range(7):
    X_list.append(c_df_log_list[i].drop(['Total Traffic Volume (MB)','cluster_id'],axis=1).values)
    y_list.append(c_df_log_list[i]['Total Traffic Volume (MB)'].values)
    scX_list.append(StandardScaler())
    scy_list.append(StandardScaler())
    X_list[i] = scX_list[i].fit_transform(X_list[i])
    y_list[i] = scy_list[i].fit_transform(y_list[i].reshape(-1,1))
    X_train,X_test,y_train,y_test=train_test_split(X_list[i],y_list[i],test_size=0.3, random_state=42)
    X_train_list.append(X_train)
    X_test_list.append(X_test)
    y_train_list.append(y_train)
    y_test_list.append(y_test)

using boosting method

In [10]:
#gcv_list = []
y_pred_list = []
r2_score_list = []
param = {
        'learning_rate':[0.1,0.05,0.01],
        'boosting_type':['gbdt'],
        'metric':['mse'],
        'num_leaves':[30,50,70],
        }


for i in range(7):
    #lgb_regressor = lgb.LGBMRegressor(n_estimators=2000)
    #gcv_list.append(GridSearchCV(lgb_regressor,param,cv=5))
    #gcv_list[i].fit(X_train_list[i],y_train_list[i])
    y_pred_list.append(gcv_list[i].best_estimator_.predict(X_test_list[i]))
    r2_score_list.append(r2_score(y_test_list[i],y_pred_list[i]))

In [11]:
r2_score_list

[0.5232074809158624,
 0.7050334342377316,
 0.5687090084498791,
 0.6722930666860876,
 0.7390917534539746,
 -0.036127162378045696,
 0.5604258906594823]

using decision tree regressor

In [ ]:
model_list = []
y_pred_list = []
r2_score_list = []
for i in range(7):
    model_list.append(DecisionTreeRegressor(min_samples_leaf=10,min_samples_split=5,max_depth=30))
    model_list[i].fit(X_train_list[i],y_train_list[i])
    y_pred_list.append(model_list[i].predict(X_test_list[i]))
    r2_score_list.append(r2_score(y_test_list[i],y_pred_list[i]))

using linear regression

In [33]:
from sklearn.linear_model import LinearRegression
model_list = []
y_pred_list = []
r2_score_list = []
for i in range(7):
    model_list.append(LinearRegression())
    model_list[i].fit(X_train_list[i],y_train_list[i])
    y_pred_list.append(model_list[i].predict(X_test_list[i]))
    r2_score_list.append(r2_score(y_test_list[i],y_pred_list[i]))

In [34]:
r2_score_list

[0.34833420301785434,
 0.49940518893568064,
 0.49143003317965306,
 0.5338046039885782,
 0.41734778065357736,
 -0.03946031433732533,
 0.3677118227084092]

linear regression with feature selection

In [59]:
import statsmodels.formula.api as sm
def backwardElimination2(x,y, SL):
    numVars = len(x[0])
    temp = np.zeros((x.shape[0],x.shape[1])).astype(int)
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        adjR_before = regressor_OLS.rsquared_adj.astype(float)
        if maxVar > SL:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    temp[:,j] = x[:, j]
                    x = np.delete(x, j, 1)
                    tmp_regressor = sm.OLS(y, x).fit()
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)
                    if (adjR_before >= adjR_after):
                        x_rollback = np.hstack((x, temp[:,[0,j]]))
                        x_rollback = np.delete(x_rollback, j, 1)
                        print (regressor_OLS.summary())
                        return x_rollback
                    else:
                        continue
    print(regressor_OLS.summary())
    return x
 
SL = 0.05

In [57]:
import statsmodels.formula.api as sm
def backwardElimination1(x,y, sl):
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(y, x).fit()
        maxVar = max(regressor_OLS.pvalues).astype(float)
        if maxVar > sl:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):
                    x = np.delete(x, j, 1)
    print(regressor_OLS.summary())
    return x
 
SL = 0.05

In [63]:
c_id=2
X = np.append(arr = np.ones((X_train_list[c_id].shape[0], 1)).astype(int), values = X_train_list[c_id], axis = 1)
y=y_train_list[c_id]
X_Modeled = backwardElimination2(X,y, SL)

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.481
Model:                            OLS   Adj. R-squared:                  0.481
Method:                 Least Squares   F-statistic:                     5328.
Date:                Mon, 29 Oct 2018   Prob (F-statistic):               0.00
Time:                        11:17:46   Log-Likelihood:                -63255.
No. Observations:               57591   AIC:                         1.265e+05
Df Residuals:                   57580   BIC:                         1.266e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0012      0.003      0.410      0.6

feature correlation

In [26]:
corr_list = []
for i in range(7):
    corr = []
    for j in range(X_list[i].shape[1]):
        corr.append(np.corrcoef(X_list[i][:,j],X_list[i][:,8])[0][1])
    corr = np.array(corr)
    corr_list.append(corr)

# Bring in the PA

In [1]:
import fiona
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
pa = fiona.open("../files/shape files/RNC_PA_region.shp")

In [30]:
pa_poly = [Polygon(item['geometry']['coordinates'][0]) for item in pa]

In [36]:
from tqdm import tqdm_notebook

for i in tqdm_notebook(range(7)):

    target_rsrp = -80
    rsrp_arr = np.zeros((10,))
    rsrp_arr[8] = target_rsrp
    rsrp_arr_tran = scX_list[i].transform(rsrp_arr.reshape(1,-1))[0]

    pred_traff=[]
    for j in range(X_list[i].shape[0]):

        num_feat = X_list[i][j,:10]

        diff_sc = rsrp_arr_tran[8] - num_feat[8]

        new_num_feat = num_feat + diff_sc*corr

        pred_traff.append(scy.inverse_transform(model_lgb.predict(new_num_feat.reshape(1,-1)))[0])

    c_df_log_list[i]['pred_traffic'] = pred_traff
    c_df_log_list[i]['pred_improved_traffic'] = c_df_log_list[i]['pred_traffic'] - c_df_log_list[i]['Total Traffic Volume (MB)']

In [43]:
c_df_log = pd.concat([c_df_log_list[0], c_df_log_list[1], c_df_log_list[2], c_df_log_list[3],
                     c_df_log_list[4], c_df_log_list[5], c_df_log_list[6]],axis=0)

In [44]:
c_df_log.sort_index(inplace=True)
c_df_log = pd.concat([c_df_log,coor_df],axis=1)
print('master df shape:',c_df_log.shape)
print('lat lng df shape:',coor_df.shape)

master df shape: (325302, 17)
lat lng df shape: (325302, 3)


In [45]:
c_df_log.head()

,ln_Number of Subscribers,ln_Average DL Throughput (kbit/s),Average UL Throughput (kbit/s),ln_DL IBLER (%),ln_UL IBLER (%),DL MCS (only for Initial Transmission),UL MCS (only for Initial Transmission),ln_UL Spectrum Efficiency (bps/HZ),RSRP(All MRs) (dBm),ln_MR Count,Total Traffic Volume (MB),cluster_id,pred_traffic,pred_improved_traffic,Latitude,Longitude,cluster_id
0,3.258097,8.668722,1023.061,2.604170,3.148882,8.0,17.0,-1.427116,-115.06,5.313206,67.130,0,18313.967942,18246.837942,13.587841,100.549038,0
1,3.912023,8.508687,1409.271,2.730464,3.065725,8.0,15.0,-1.171183,-114.87,6.416732,218.492,0,23348.110106,23129.618106,13.587841,100.549488,0
2,3.044522,8.615236,1471.403,2.679651,3.360723,7.0,16.0,-1.272966,-114.08,5.429346,109.247,0,19545.125447,19435.878447,13.588740,100.549038,0
3,0.693147,8.216814,865.221,2.989211,2.311545,5.0,16.0,-2.207275,-113.83,4.795791,3.864,0,2998.125037,2994.261037,13.950268,100.900223,0
4,3.610918,8.487582,1314.256,2.632608,3.216473,7.0,12.0,-1.347074,-113.01,5.981414,144.107,0,16380.262202,16236.155202,13.588291,100.549038,0


In [47]:
from tqdm import tqdm
tqdm.pandas()

def map_coor2cid(x):
    point = Point(x['Longitude'],x['Latitude'])
    for i,pa in enumerate(pa_poly):
        if pa.contains(point):
            return i
        
c_df_log['pa_id'] = c_df_log.progress_apply(map_coor2cid,axis=1)

100%|██████████| 325302/325302 [13:47<00:00, 393.14it/s]


In [49]:
pa_df = pd.DataFrame(c_df_log.groupby('pa_id')['pred_improved_traffic'].sum()).reset_index()

In [50]:
# find each pa center
pa_lng_center = []
pa_lat_center = []
for item in tqdm(pa):
    lng_sum = 0
    lat_sum = 0
    for coor in item['geometry']['coordinates'][0]:
        lng_sum+=coor[0]
        lat_sum+=coor[1]
    
    pa_lng_center.append(lng_sum/len(item['geometry']['coordinates'][0]))
    pa_lat_center.append(lat_sum/len(item['geometry']['coordinates'][0]))

100%|██████████| 450/450 [00:00<00:00, 29340.83it/s]


In [51]:
pa_df_2 = pd.DataFrame({'pa_id':range(450),'lng':pa_lng_center,'lat':pa_lat_center})

In [52]:
pa_df = pd.merge(pa_df,pa_df_2,on='pa_id')

In [53]:
pa_df.head()

,pa_id,pred_improved_traffic,lng,lat
0,2.0,3.139532e+03,100.765343,14.038855
1,3.0,8.149776e+04,100.716572,13.940437
2,4.0,2.145318e+06,100.719748,13.921862
3,5.0,2.587559e+04,100.736650,13.940515
4,7.0,1.371955e+05,100.711425,13.869243


In [13]:
pa[99]['geometry']['coordinates'][0]

[(100.645691, 13.630703),
 (100.646016, 13.632427),
 (100.647403, 13.635867),
 (100.651437, 13.634104),
 (100.652168, 13.635709),
 (100.653095, 13.637452),
 (100.660523, 13.634943),
 (100.660631, 13.635179),
 (100.662316, 13.634515),
 (100.662134, 13.634153),
 (100.660387, 13.634837),
 (100.660177, 13.634515),
 (100.659587, 13.634714),
 (100.656821, 13.635646),
 (100.65501, 13.630921),
 (100.657528, 13.630079),
 (100.657122, 13.629237),
 (100.650597, 13.631657),
 (100.650327, 13.631789),
 (100.648344, 13.630467),
 (100.647545, 13.631256),
 (100.646449, 13.630283),
 (100.64565, 13.62977),
 (100.645326, 13.630059),
 (100.645691, 13.630703)]

In [2]:
import pandas as pd
#pa_df.to_csv('pa_traffic.csv',index=False)
pa_df = pd.read_csv('pa_traffic.csv')

plotting the bubble map

In [55]:
pa_df['size'] = (pa_df['pred_improved_traffic'] - pa_df['pred_improved_traffic'].min())/(pa_df['pred_improved_traffic'].max() - pa_df['pred_improved_traffic'].min())*50

from bokeh.io import output_file,show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool

plot = figure(plot_width=1000,title='predicted traffic gain(MB) from coverage improvement of each PA(Problem Area)')
source = ColumnDataSource(pa_df)
plot.circle('lng','lat',source=source,size='size',fill_alpha=0.2)
hover = HoverTool(tooltips=[('pa id', '@pa_id'),('predicted traffic gain (MB)', '@pred_improved_traffic')])
plot.add_tools(hover)

output_file('pa.html')
show(plot)

plotting the pa polygons

In [3]:
pa_df['pa_id'] = pa_df['pa_id'].astype(int)

pred_traff = pa_df['pred_improved_traffic'].values

import numpy as np
norm_pred_traff = (pred_traff - np.min(pred_traff))/(np.max(pred_traff)-np.min(pred_traff))

lng_list = []
lat_list = []
alpha_list = []
for i,pa_id in enumerate(pa_df['pa_id']):
    lng=[]
    lat=[]
    for coor in pa[pa_id]['geometry']['coordinates'][0]:
        lng.append(coor[0])
        lat.append(coor[1])
    lng_list.append(lng)
    lat_list.append(lat)
    alpha_list.append(norm_pred_traff[i])

from bokeh.io import output_file,show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource
from bokeh.models import HoverTool

source = ColumnDataSource(data = dict(
    pa_id = pa_df['pa_id'].values,
    lng = lng_list,
    lat = lat_list,
    al = alpha_list,
    traff = pred_traff
))
plot = figure(plot_width=1000,title='predicted traffic gain(MB) from coverage improvement of each PA(Problem Area)')
plot.patches('lng','lat',alpha='al',source=source)

hover = HoverTool(tooltips=[('pa id', '@pa_id'),('predicted traffic gain (MB)', '@traffic_gain')])
plot.add_tools(hover)

output_file('pa2.html')
show(plot)